In [39]:
import os
import csv
import math as m
import numpy as np
import pandas as pd 
import subprocess
import sys
import re
import time
import datetime
from datetime import timedelta

from pathlib import Path

# sys.path.append(r"C:\Users\June Means\.conda\envs\CVenv\Lib\site-packages\cv2") # This seems to be an issue uniquely with my conda installation
# sys.path.append(r"C:\Users\June Means\.conda\envs\CVenv\Lib\site-packages\openpyxl")
import cv2
import struct
import threading
from collections import deque

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.backends.backend_agg import FigureCanvasAgg

In [40]:
FFMPEG_PATH= r"C:\Program Files\ffmpeg-8.0-full_build\bin\ffmpeg.exe" # The path of the FFMPEG.exe on the system
FFPROBE_PATH= r"C:\Program Files\ffmpeg-8.0-full_build\bin\ffprobe.exe"

In [44]:
def ffmpeg_frame_probe(video_path):
    """
    Probe a video file to get its frames per second (fps) using ffprobe.
    
    Args:
        video_path: Path to video file
        
    Returns:
        float: The fps of the video
    """
    cmd = [
        "ffprobe",
        "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "stream=r_frame_rate,duration",  # Get frame rate and duration
        "-of", "default=noprint_wrappers=1:nokey=1",
        video_path
    ]
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        # ffprobe returns frame rate as a fraction (e.g., "30000/1001" or "30/1")
        print(result)
        # fps_str = result.stdout.strip()
        fps_str = result.stdout.splitlines()[0].strip()
        duration_str = result.stdout.splitlines()[1].strip()
        # Parse the fraction
        if '/' in fps_str:
            numerator, denominator = fps_str.split('/')
            fps = float(numerator) / float(denominator)
        else:
            fps = float(fps_str)

        duration = float(duration_str)
        return fps, duration
        
    except subprocess.CalledProcessError as e:
        print(f"Error during ffprobe execution: {e}")
        print(f"stderr: {e.stderr}")
        print("A assumed fps of 29.595 has been returned. Duration cannot be assumed and so 0 is returned")
        return(29.595, 0)

print(ffmpeg_frame_probe(r"C:\Users\jjmmc\Downloads\TRAP2-5-208-L3-output_with_overlaycollisions_id1.mp4"))

# class video_preprocessing():
    # def __init__(self):
    #     self.var =0
def animal_cropper(processing_video_path, frame_span = 3600):
    """
    This removes the animal from a video to obtain an average that should reflect the background of the video
    This works by masking out the are of highest movement and
    """
    # sample a spaced out number of frames 
    # Sample every 30 frames (adjust based on your video)
    cap = cv2.VideoCapture(processing_video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video")
    else:
        print(f"Video opened successfully")
        print(f"Total frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")
        print(f"FPS: {cap.get(cv2.CAP_PROP_FPS)}")
    fps, duration = ffmpeg_frame_probe(processing_video_path)
    
    sampled_frames = []
    frame_idx = 0
    
    if duration*fps < frame_span:
        print(f"This video is less than {frame_span} frames long. Thus there may be too few samples to properly construct the background.")
        sample_interval = int((duration*fps)/30)
    else:
        sample_interval = 30
    
    while cap.isOpened() and frame_idx < frame_span:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_idx % sample_interval == 0:
            sampled_frames.append(frame)        
        frame_idx += 1
    
    cap.release()
    
    print(f"\nTotal sampled frames: {len(sampled_frames)}")
    
    if len(sampled_frames) == 0:
        print("ERROR: No frames were sampled!")
    else:
        print(f"Frame shape: {sampled_frames[0].shape}")
        print(f"Frame dtype: {sampled_frames[0].dtype}")
        
        # Convert to numpy array first
        frames_array = np.array(sampled_frames)
        print(f"Array shape: {frames_array.shape}")
        
        # Compute median
        background = np.median(frames_array, axis=0).astype(np.uint8)
        background = np.mean(frames_array, axis=0).astype(np.uint8)
        print(f"Background shape: {background.shape}")
        print(f"Background dtype: {background.dtype}")
        print(f"Background min/max: {background.min()}/{background.max()}")
        
        # Check for NaN
        if np.isnan(background).any():
            print("WARNING: Background contains NaN values!")
        else:
            success = cv2.imwrite('output_image.jpeg', background)
            print(f"Image write success: {success}")

def roi_finder(processing_video_path, threshold1 =100, threshold2= 200):
    """
    This uses edge detection to find the dimensions of the box, the location of the nest and of the loom. If it fails to find a good fit it will change threshold parameters until it gives up
    """
    low_thresh = np.arange(20,180, 20)
    high_thresh = np.arange(40,200, 20)

    # for low,high in zip(low_thresh, high_thresh):
        
    #     # image = cv2.imread(processing_video_path)
    #     # edges = cv2.Canny(image, low, high)
        
        
        
    #     success = cv2.imwrite(f'output_edges{low}-{high}.jpeg', edges)
    #     print(f"Image write success: {success}")

    # for i in range(1,7,2):
    #     image = cv2.imread(processing_video_path, cv2.IMREAD_GRAYSCALE)
    #     edges =cv2.Laplacian(image, -1, ksize=i)
    #     success = cv2.imwrite(f'output_edges_{i}.jpeg', edges)
        # success = cv2.imwrite(f'output_edges{low}-{high}.jpeg', edges)
    
    image = cv2.imread(processing_video_path)
    blur = cv2.GaussianBlur(image, (5, 5), 1.4)
    edges = cv2.Canny(blur, 20, 40)        
        
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and pick the largest one (assuming it's your target rectangle)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        print(len(contours))
        rect = cv2.minAreaRect(largest_contour)
        
        box = cv2.boxPoints(rect)
        box = np.int32(box)
        
        image_bgr = image.copy()
        cv2.drawContours(image_bgr, [box], 0, (0, 0, 255), 2)
        
        cv2.imshow('Fitted Rectangle', image_bgr)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    # This is a bit mathy but one of the things that doesn't change as a result of perspective shifts or even 



# animal_cropper(r"C:\Users\jjmmc\Downloads\20250113_socialTRAP2_5VLS_208+209.mp4")
animal_cropper(r"C:\Users\jjmmc\Downloads\20241101_TRAP2social_5VLS_M_L5_TRAP2156+157.mp4")
# roi_finder(r"C:\Users\jjmmc\Documents\FileTreeShenanigans\output_image.jpeg")


def angle_correction(inter_video_path):
    #check if the box angle
    angle=30
    # ffmpeg -i input.mp4 -vf "rotate=angle*PI/180" output.mp4

    cmd = [
        "ffmpeg",
        "-v", "error",  # Hide unnecessary output
        "-i", inter_video_path,
        "-vf",
        f"rotate={angle}*PI/180",
        "output_temp.mp4",
        # "&&", "mv", #Then if successful move the output to the input
        # "output_temp.mp4", inter_video_path
    ]
    print(cmd)
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        # ffprobe returns frame rate as a fraction (e.g., "30000/1001" or "30/1")
        
        
        # Parse the fraction
        # if '/' in fps_str:
        #     numerator, denominator = fps_str.split('/')
        #     fps = float(numerator) / float(denominator)
        # else:
        #     fps = float(fps_str)
            
        # return fps
        
    except subprocess.CalledProcessError as e:
        print(f"Error during ffprobe execution: {e}")
        print(f"stderr: {e.stderr}")
        print("A assumed fps of 29.595 has been returned")
        return(29.595)

def show_roi():
    """
    This is a visualization of the ROIs on the first frame of the video. 
    """
    

# angle_correction(r"C:\Users\jjmmc\Downloads\TRAP2-5-208-L3-output_with_overlaycollisions_id1.mp4")



CompletedProcess(args=['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=r_frame_rate,duration', '-of', 'default=noprint_wrappers=1:nokey=1', 'C:\\Users\\jjmmc\\Downloads\\TRAP2-5-208-L3-output_with_overlaycollisions_id1.mp4'], returncode=0, stdout='29583/1000\n9.938140\n', stderr='')
(29.583, 9.93814)
Video opened successfully
Total frames: 25137
FPS: 29.59517818093909
CompletedProcess(args=['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=r_frame_rate,duration', '-of', 'default=noprint_wrappers=1:nokey=1', 'C:\\Users\\jjmmc\\Downloads\\20241101_TRAP2social_5VLS_M_L5_TRAP2156+157.mp4'], returncode=0, stdout='60/1\n849.361300\n', stderr='')

Total sampled frames: 120
Frame shape: (1080, 1920, 3)
Frame dtype: uint8
Array shape: (120, 1080, 1920, 3)
Background shape: (1080, 1920, 3)
Background dtype: uint8
Background min/max: 0/255
Image write success: True


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Optional, List
from dataclasses import dataclass

@dataclass
class RotatedBoxROI:
    """Container for detected rotated box region of interest."""
    corners: np.ndarray
    center: Tuple[float, float]
    width: float
    height: float
    angle: float
    confidence: float


def detect_box_negative_space(image: np.ndarray, 
                               min_area_ratio: float = 1/6,
                               visualize: bool = False) -> Optional[RotatedBoxROI]:
    """
    Negative space approach: Find smooth regions enclosed by Hough lines.
    
    Strategy:
    1. Detect all lines with Hough transform
    2. Draw lines on a blank canvas to create enclosed regions
    3. Find large smooth regions (the box interior)
    4. Use the lines that border these regions as the box edges
    5. Construct bounding box from border lines
    
    Args:
        image: BGR image
        min_area_ratio: Minimum box area as fraction of image area
        visualize: Whether to save visualizations
    
    Returns:
        RotatedBoxROI object or None if detection fails
    """
    h, w = image.shape[:2]
    
    # Preprocessing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    # Gaussian blur
    blur = cv2.GaussianBlur(enhanced, (5, 5), 1.4)
    
    # Canny edge detection
    edges = cv2.Canny(blur, 30, 50, apertureSize=3)
    cv2.imshow('canny edge detection', edges)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Dilate edges slightly to connect nearby segments
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    
    # Hough line detection
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50,
                           minLineLength=60, maxLineGap=20)
    
    if lines is None:
        print("No lines detected!")
        return None
    
    print(f"Detected {len(lines)} lines")
    
    # Filter short lines and store line properties
    lines_data = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        
        if length < 80:
            continue
        
        lines_data.append({
            'line': line[0],
            'length': length
        })
    
    print(f"Kept {len(lines_data)} lines after filtering")
    
    if len(lines_data) < 4:
        print("Insufficient lines")
        return None
    
    # Create a canvas with all lines drawn
    line_canvas = np.zeros((h, w), dtype=np.uint8)
    
    for line_data in lines_data:
        x1, y1, x2, y2 = line_data['line']
        cv2.line(line_canvas, (x1, y1), (x2, y2), 255, 2)
    
    if visualize:
        cv2.imwrite('/home/claude/line_canvas.png', line_canvas)
    
    # Invert to get negative space (regions enclosed by lines)
    negative_space = cv2.bitwise_not(line_canvas)
    
    # Find connected components (regions)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        negative_space, connectivity=8
    )
    
    print(f"Found {num_labels - 1} regions in negative space")
    
    # Analyze regions to find smooth areas (potential box interior)
    # The box interior should be:
    # 1. Large area (meets min_area_ratio)
    # 2. Located in central portion of image
    # 3. Has smooth texture (low variance in original image)
    
    img_area = h * w
    candidate_regions = []
    
    for label in range(1, num_labels):  # Skip background (label 0)
        area = stats[label, cv2.CC_STAT_AREA]
        
        # Must be at least min_area_ratio of image
        if area < min_area_ratio * img_area:
            continue
        
        # Must not be too large (avoid selecting entire frame)
        if area > 0.9 * img_area:
            continue
        
        # Create mask for this region
        region_mask = (labels == label).astype(np.uint8) * 255
        
        # Calculate smoothness (inverse of variance in the region)
        region_pixels = gray[region_mask > 0]
        if len(region_pixels) > 0:
            variance = np.var(region_pixels)
            smoothness = 1.0 / (1.0 + variance / 100.0)
        else:
            smoothness = 0
        
        # Calculate centrality (how close to image center)
        cx, cy = centroids[label]
        center_dist = np.sqrt((cx - w/2)**2 + (cy - h/2)**2)
        max_dist = np.sqrt((w/2)**2 + (h/2)**2)
        centrality = 1.0 - (center_dist / max_dist)
        
        # Combined score
        area_score = min(area / (0.5 * img_area), 1.0)
        score = 0.4 * area_score + 0.3 * smoothness + 0.3 * centrality
        
        candidate_regions.append({
            'label': label,
            'area': area,
            'mask': region_mask,
            'variance': variance,
            'smoothness': smoothness,
            'centrality': centrality,
            'score': score,
            'centroid': (cx, cy)
        })
    
    if not candidate_regions:
        print("No candidate regions found")
        return None
    
    # Sort by score and take the best
    candidate_regions.sort(key=lambda x: x['score'], reverse=True)
    
    print(f"\nTop candidate regions:")
    for i, region in enumerate(candidate_regions[:3]):
        print(f"  {i+1}. Area: {region['area']/img_area*100:.1f}%, "
              f"Variance: {region['variance']:.1f}, "
              f"Score: {region['score']:.2f}")
    
    best_region = candidate_regions[0]
    region_mask = best_region['mask']
    
    print(f"\nSelected region: {best_region['area']/img_area*100:.1f}% of frame")
    
    # Find the lines that border this region
    # Dilate the region slightly and subtract to get boundary
    kernel_dilate = np.ones((5, 5), np.uint8)
    dilated_region = cv2.dilate(region_mask, kernel_dilate, iterations=1)
    boundary = cv2.subtract(dilated_region, region_mask)
    
    # Find which lines intersect with this boundary
    border_lines = []
    
    for line_data in lines_data:
        x1, y1, x2, y2 = line_data['line']
        
        # Create a temporary line mask
        temp_line = np.zeros((h, w), dtype=np.uint8)
        cv2.line(temp_line, (x1, y1), (x2, y2), 255, 2)
        
        # Check overlap with boundary
        overlap = cv2.bitwise_and(temp_line, boundary)
        overlap_pixels = np.sum(overlap > 0)
        
        # If line has significant overlap with boundary, it's a border line
        if overlap_pixels > 10:  # At least 10 pixels overlap
            border_lines.append(line_data)
    
    print(f"Found {len(border_lines)} lines bordering the region")
    
    if len(border_lines) < 4:
        print("Insufficient border lines")
        return None
    
    # Cluster border lines by angle
    angles = []
    for line_data in border_lines:
        x1, y1, x2, y2 = line_data['line']
        angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
        angle = angle % 180
        angles.append(angle)
        line_data['angle'] = angle
    
    # Simple angle clustering with 10 degree bins
    angle_bins = {}
    bin_size = 10.0
    
    for line_data in border_lines:
        angle = line_data['angle']
        bin_id = int(angle / bin_size)
        
        if bin_id not in angle_bins:
            angle_bins[bin_id] = []
        angle_bins[bin_id].append(line_data)
    
    # Find two largest clusters
    cluster_sizes = [(bin_id, len(lines)) for bin_id, lines in angle_bins.items()]
    cluster_sizes.sort(key=lambda x: x[1], reverse=True)
    
    if len(cluster_sizes) < 2:
        print("Need at least 2 angle clusters in border lines")
        return None
    
    print(f"Border lines form {len(cluster_sizes)} angle clusters")
    
    # Try to find perpendicular clusters
    best_box = None
    best_confidence = 0
    
    for i in range(min(3, len(cluster_sizes))):
        for j in range(i+1, min(4, len(cluster_sizes))):
            bin1_id, size1 = cluster_sizes[i]
            bin2_id, size2 = cluster_sizes[j]
            
            cluster1 = angle_bins[bin1_id]
            cluster2 = angle_bins[bin2_id]
            
            angle1_avg = np.mean([l['angle'] for l in cluster1])
            angle2_avg = np.mean([l['angle'] for l in cluster2])
            
            angle_diff = abs(angle1_avg - angle2_avg)
            if angle_diff > 90:
                angle_diff = 180 - angle_diff
            
            # Check perpendicularity
            if not (70 < angle_diff < 110):
                continue
            
            print(f"\nTrying border clusters {i},{j}: {size1} + {size2} lines, "
                  f"angles {angle1_avg:.1f}° and {angle2_avg:.1f}° (diff: {angle_diff:.1f}°)")
            
            # Instead of using all border lines, find the extreme lines in each cluster
            # that bound the smooth region
            
            img_center = np.array([w / 2, h / 2])
            
            # For each cluster, find the lines that are furthest from the center
            # in the perpendicular direction to the cluster's angle
            def get_distance_from_center(line_data):
                """Get distance of line midpoint from image center."""
                x1, y1, x2, y2 = line_data['line']
                mid = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
                return np.linalg.norm(mid - img_center)
            
            # For cluster 1, get perpendicular direction
            perp_angle1 = (angle1_avg + 90) % 180
            perp_rad1 = np.radians(perp_angle1)
            perp_vec1 = np.array([np.cos(perp_rad1), np.sin(perp_rad1)])
            
            # Get signed distances for cluster 1 lines
            signed_dists1 = []
            for line_data in cluster1:
                x1, y1, x2, y2 = line_data['line']
                mid = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
                vec_to_mid = mid - img_center
                signed_dist = np.dot(vec_to_mid, perp_vec1)
                signed_dists1.append((line_data, signed_dist))
            
            signed_dists1.sort(key=lambda x: x[1])
            edge1_a = signed_dists1[0][0]['line']   # Most negative
            edge1_b = signed_dists1[-1][0]['line']  # Most positive
            
            # For cluster 2, same approach
            perp_angle2 = (angle2_avg + 90) % 180
            perp_rad2 = np.radians(perp_angle2)
            perp_vec2 = np.array([np.cos(perp_rad2), np.sin(perp_rad2)])
            
            signed_dists2 = []
            for line_data in cluster2:
                x1, y1, x2, y2 = line_data['line']
                mid = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
                vec_to_mid = mid - img_center
                signed_dist = np.dot(vec_to_mid, perp_vec2)
                signed_dists2.append((line_data, signed_dist))
            
            signed_dists2.sort(key=lambda x: x[1])
            edge2_a = signed_dists2[0][0]['line']
            edge2_b = signed_dists2[-1][0]['line']
            
            # Find intersections of these 4 edge lines
            def line_intersection(line1, line2):
                """Find intersection of two lines."""
                x1, y1, x2, y2 = line1
                x3, y3, x4, y4 = line2
                
                denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
                
                if abs(denom) < 1e-6:
                    return None
                
                t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denom
                
                x = x1 + t * (x2 - x1)
                y = y1 + t * (y2 - y1)
                
                return np.array([x, y])
            
            # Find all 4 corners
            corners = []
            for e1 in [edge1_a, edge1_b]:
                for e2 in [edge2_a, edge2_b]:
                    corner = line_intersection(e1, e2)
                    if corner is not None:
                        corners.append(corner)
            
            if len(corners) != 4:
                print(f"  Got {len(corners)} corners, need 4")
                continue
            
            corners = np.array(corners, dtype=np.float32)
            
            # Get rect from corners
            rect = cv2.minAreaRect(corners)
            center, (width, height), angle_rot = rect
            box_points = cv2.boxPoints(rect)
            
            area = width * height
            area_ratio = area / img_area
            
            print(f"  Box area: {area_ratio*100:.1f}% of frame")
            
            # Validate
            if area_ratio < min_area_ratio or area_ratio > 0.9:
                continue
            
            # Calculate confidence
            area_score = min(area_ratio / 0.4, 1.0)
            angle_score = 1.0 - abs(angle_diff - 90) / 20
            region_score = best_region['smoothness']
            
            confidence = 0.3 * area_score + 0.3 * angle_score + 0.4 * region_score
            
            print(f"  Confidence: {confidence:.2f}")
            
            if confidence > best_confidence:
                best_confidence = confidence
                best_box = {
                    'corners': box_points,
                    'rect': rect,
                    'border_lines': cluster1 + cluster2,
                    'edge_lines': [edge1_a, edge1_b, edge2_a, edge2_b],
                    'region_mask': region_mask
                }
    
    if best_box is None:
        print("\nNo valid box found from border lines")
        return None
    
    # Extract results
    corners = best_box['corners']
    rect = best_box['rect']
    center, (width, height), angle = rect
    confidence = best_confidence
    
    area = width * height
    area_ratio = area / img_area
    
    print(f"\n✓ Detection successful!")
    print(f"  Confidence: {confidence:.2f}")
    print(f"  Center: ({center[0]:.1f}, {center[1]:.1f})")
    print(f"  Dimensions: {width:.1f} x {height:.1f}")
    print(f"  Rotation: {angle:.1f}°")
    print(f"  Area: {area_ratio*100:.1f}% of frame")
    
    # Visualization
    if visualize:
        fig, axes = plt.subplots(2, 3, figsize=(24, 16))
        
        # 1. Original image
        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
        axes[0, 0].axis('off')
        
        # 2. Line canvas
        axes[0, 1].imshow(line_canvas, cmap='gray')
        axes[0, 1].set_title('All Detected Lines', fontsize=14, fontweight='bold')
        axes[0, 1].axis('off')
        
        # 3. Negative space with regions
        colored_regions = cv2.applyColorMap(
            (labels * (255 // num_labels)).astype(np.uint8), 
            cv2.COLORMAP_JET
        )
        axes[0, 2].imshow(colored_regions)
        axes[0, 2].set_title(f'Negative Space Regions ({num_labels-1} found)', 
                            fontsize=14, fontweight='bold')
        axes[0, 2].axis('off')
        
        # 4. Selected region
        axes[1, 0].imshow(best_box['region_mask'], cmap='gray')
        axes[1, 0].set_title(f'Selected Smooth Region ({area_ratio*100:.1f}% of frame)', 
                            fontsize=14, fontweight='bold')
        axes[1, 0].axis('off')
        
        # 5. Border lines with edge lines highlighted
        img_border = image.copy()
        # Draw all border lines in yellow
        for line_data in best_box['border_lines']:
            x1, y1, x2, y2 = line_data['line']
            cv2.line(img_border, (x1, y1), (x2, y2), (0, 255, 255), 2)
        # Draw the 4 selected edge lines in green
        for edge_line in best_box['edge_lines']:
            x1, y1, x2, y2 = edge_line
            cv2.line(img_border, (x1, y1), (x2, y2), (0, 255, 0), 4)
        axes[1, 1].imshow(cv2.cvtColor(img_border, cv2.COLOR_BGR2RGB))
        axes[1, 1].set_title(f'Border Lines + 4 Edge Lines (green)', 
                            fontsize=14, fontweight='bold')
        axes[1, 1].axis('off')
        
        # 6. Final detection
        img_final = image.copy()
        cv2.polylines(img_final, [corners.astype(np.int32)], True, (0, 255, 0), 4)
        
        for i, corner in enumerate(corners):
            cv2.circle(img_final, tuple(corner.astype(int)), 12, (0, 0, 255), -1)
            cv2.putText(img_final, str(i+1), tuple((corner - [5, -5]).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        cv2.putText(img_final, f'Confidence: {confidence:.2f}', 
                   (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
        cv2.putText(img_final, f'Angle: {angle:.1f} deg', 
                   (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
        cv2.putText(img_final, f'Area: {area_ratio*100:.1f}% of frame', 
                   (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
        
        axes[1, 2].imshow(cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB))
        axes[1, 2].set_title('Final Detection', fontsize=14, fontweight='bold')
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        plt.savefig('negative_space_detection.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        print("\n✓ Visualization saved to negative_space_detection.png")
    
    box_roi = RotatedBoxROI(
        corners=corners,
        center=center,
        width=width,
        height=height,
        angle=angle,
        confidence=confidence
    )
    
    return box_roi


if __name__ == "__main__":
    image_path = r"C:\Users\jjmmc\Documents\FileTreeShenanigans\output_image.jpeg"
    image = cv2.imread(image_path)
    
    print("="*70)
    print("NEGATIVE SPACE DETECTION")
    print("="*70)
    
    box_roi = detect_box_negative_space(image, min_area_ratio=1/10, visualize=True)
    
    if box_roi is not None:
        print("\n" + "="*70)
        print("DETECTED CORNERS:")
        print("="*70)
        for i, corner in enumerate(box_roi.corners):
            print(f"  Corner {i+1}: ({corner[0]:.1f}, {corner[1]:.1f})")
            
            # Check if corner is out of bounds
            h, w = image.shape[:2]
            if corner[0] < 0 or corner[0] > w or corner[1] < 0 or corner[1] > h:
                print(f"    ⚠ WARNING: Corner {i+1} is outside image bounds!")
    else:
        print("\n✗ Detection failed")

NEGATIVE SPACE DETECTION
Detected 5982 lines
Kept 4219 lines after filtering
Found 7912 regions in negative space

Top candidate regions:
  1. Area: 14.3%, Variance: 522.7, Score: 0.39

Selected region: 14.3% of frame
Found 197 lines bordering the region
Border lines form 18 angle clusters

Trying border clusters 0,3: 50 + 16 lines, angles 174.7° and 87.6° (diff: 87.1°)
  Box area: 13.4% of frame
  Confidence: 0.42

Trying border clusters 1,3: 21 + 16 lines, angles 4.0° and 87.6° (diff: 83.6°)
  Box area: 16.3% of frame
  Confidence: 0.39

✓ Detection successful!
  Confidence: 0.42
  Center: (1069.5, 761.7)
  Dimensions: 508.9 x 546.7
  Rotation: 86.0°
  Area: 13.4% of frame

✓ Visualization saved to negative_space_detection.png

DETECTED CORNERS:
  Corner 1: (779.1, 526.9)
  Corner 2: (1324.5, 488.9)
  Corner 3: (1359.8, 996.5)
  Corner 4: (814.5, 1034.5)


In [ ]:
def trim_video(input_path, output_path, start_time, end_time): 
    """
    Trims a video using ffmpeg from start_time to end_time. This could be modified for other video processing needs 

    :param input_path: Path to the input video file.
    :param output_path: Desired output file name.
    :param start_time: Start timestamp (e.g., "00:01:00").
    :param end_time: End timestamp (e.g., "00:02:00").
    """
    cmd = [
        FFMPEG_path,
        "-y", ##Overwrite
        "-ss", str(start_time),
        "-to", str(end_time),
        "-i", input_path,
        # "-c", "copy",  # This is tell ffmpeg to not reencode the video. This seems to be causing our import issue with sleap
        "-vf", "crop=800:ih:720:0", # This crops the video essentially replacing the need to use handbrake. Cropping is done in a w:h:x:y format where w is width, h is height and x and y describe horizontal and vertical offsets respectively
        output_path
    ]
    print(cmd)
    TEST = [r"C:\Users\June Means\AppData\Local\ffmpeg-2025-06-28-git-cfd1f81e7d-full_build\bin\ffmpeg.exe", "-version"]
    subprocess.run(TEST, check=True)
    try:
        subprocess.run(cmd, check=True)
        print(f"Trimmed video saved to: {output_path}")
    except subprocess.CalledProcessError as e:
        print("Error during ffmpeg execution:", e)


In [1]:
"""
Negative Space ROI Detection for Behavioral Arenas

Detects rectangular arenas by finding smooth regions (low edge density)
surrounded by busy/textured backgrounds. Handles split arenas caused by
lighting artifacts by merging adjacent regions.

Usage:
    roi, vis = detect_arena(image, known_aspect_ratio=1.5, visualize=True)
"""

import sys
import os
import cv2
import numpy as np
from dataclasses import dataclass
from typing import Tuple, Optional, List

# The path you want to add. Use absolute paths for reliability.
new_path = R"C:\Users\jjmmc\Downloads"

# Append the new path to sys.path
sys.path.append(new_path)

from negative_space_roi import detect_arena


image_path = r"C:\Users\jjmmc\Documents\FileTreeShenanigans\output_image.jpeg"
image = cv2.imread(image_path)
roi, vis = detect_arena(
    image,
    known_aspect_ratio=1.5,  # Close to 1.47
    min_area_ratio=0.18,
    max_area_ratio=0.30,
    aspect_tolerance=0.20,
    density_threshold=20,
    visualize=True
)
if roi:
    corners = roi.corners  # 4x2 numpy array
    mask = roi.get_mask(image.shape)

print(corners)
if vis is not None:
    print("works")
    cv2.imwrite("detection_debug.jpg", vis)

[[ 735.15186   120.39032 ]
 [1356.2312     53.675934]
 [1454.019     964.03375 ]
 [ 832.9397   1030.7482  ]]
works


In [ ]:
def nest_calculator(f_pts, percent_of_perspective=0.1955, buffer=40):
    """
    This is a hard coded calculation of where the nest should be given the floor's corners. 
    The rectange's points are defined as

    0-1
    | |
    3-2
    """
    nest_rect = np.zeros((4,2))
    side_vector = f_pts[0]-f_pts[1]
    side_vector = sidevector / np.linalg.norm(side_vector)
    up_vector = np.rot90(side_vector)
    nest_rect[0] = percent_of_perspective*f_pts[0] +(1-percent_of_perspective) * f_pts[3] +side_vector*20 # Find the point ~20% from the bottom left to top left and then shift it over 20 units left perpendicularly
    nest_rect[2]= f_pts[2]- buffer*side_vector-buffer*up_vector
    nest_rect[3]= f_pts[3]+ buffer*side_vector-buffer*up_vector
    nest_rect[1]= nest_rect[0]+nest_rect[2]-nest_rect[3] #take the vector from 3 to 2 and add it to 1 to obtain the remaining point
    return(nest_rect)

def loom_calculator(f_pts, percent_of_perspective=0.1506, radius =32):
    loom_circle = np.zeros(2)
    loom_circle[1]= radius
    
    down_vector = f_pts[3]-f_pts[0]
    loom_circle[0]= 0.5*(f_pts[0]+f_pts[1])+(down_vector*percent_of_perspective)

    return(loom_circle)
    